In [2]:
import pandas as pd
import requests
import time

In [3]:
gps_inp = "../../samples/data/atuanhm_2705_gps_processed.csv"
vehicle_inp = "../../samples/data/atuanhm_2705_report_vehicle_processed.csv"

id = "sample car 4"
delay=0.00

In [4]:
tracking_gps_url = "http://localhost:1234/trackMany"
tracking_vehicle_url = "http://localhost:1234/track"

In [5]:
import json

def to_json(row, id=None):
    obj = {}
    obj['time'] = row['positionTime'] * 1000
    obj['lat'] = row['lat']
    obj['lng'] = row['lng']
    obj['azimuth'] = row['heading']
    if 'speed' in row:
        obj['velocity'] = row['speed']
    if 'height' in row:
        obj['height'] = row['height']
    if id is not None:
        obj["id"] = id
        
    return json.dumps(obj)

In [6]:
def merge_gps(series, id=None):
    res = []
    for u in range(series.shape[0]):
        res.append(json.loads(series.iloc[u]['json']))
        
    obj = {"samples": res}
    if id is not None:
        obj["id"] = id
    
    return json.dumps(obj)

In [7]:
gps_df = pd.read_csv(gps_inp)
gps_df['json'] = gps_df.apply(lambda u: to_json(u), axis=1)

In [8]:
gps_df.head()

,ts,index_record,heading,lat,lng,positionTime,h_positionTime,h_receiveTime,json
0,1622119844873,0,26,21.03023,105.78133,1622119835,2021-05-27 19:50:35,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133000000001, ""..."
1,1622119844873,1,26,21.03023,105.78133,1622119836,2021-05-27 19:50:36,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133000000001, ""..."
2,1622119844873,2,26,21.03023,105.78134,1622119837,2021-05-27 19:50:37,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133999999999, ""..."
3,1622119844873,3,26,21.03024,105.78134,1622119838,2021-05-27 19:50:38,2021-05-27 19:50:44,"{""lat"": 21.03024, ""lng"": 105.78133999999999, ""..."
4,1622119844873,4,26,21.03023,105.78134,1622119839,2021-05-27 19:50:39,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133999999999, ""..."


In [9]:
vehicle_df = pd.read_csv(vehicle_inp)
vehicle_df['json'] = vehicle_df.apply(lambda r: to_json(r, id), axis=1)
vehicle_df['type'] = 'vehicle'

### Merge

In [10]:
gps_merge = pd.DataFrame()
gps_merge = gps_df.groupby(by='ts').apply(lambda row: merge_gps(row, id))

gps_merge = gps_merge.reset_index()
gps_merge.columns = ['ts', 'json']
gps_merge['type'] = 'gps'

In [33]:
gps_merge['json'][0]

'{"id": "sample car 4", "samples": [{"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119835000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119836000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119837000, "azimuth": 26}, {"lat": 21.03024, "lng": 105.78133999999999, "time": 1622119838000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119839000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119840000, "azimuth": 26}]}'

In [34]:
merge_all = pd.concat([gps_merge, vehicle_df[['ts', 'json', 'type']]], ignore_index=True)

In [35]:
merge_all.sort_values(by='ts', inplace=True)

In [40]:
total_gps = len(merge_all[merge_all['type'] == 'gps'])* 7 + len(merge_all[merge_all['type'] == 'vehicle'])

In [55]:
start = time.time()
for i in range(merge_all.shape[0]):
    row = merge_all.iloc[i]
#     print row
    push = True
    if row['type'] == 'gps':
        url = tracking_gps_url
        push = True
    else:
        push = True
        url = tracking_vehicle_url
    
    if push:
        x = requests.post(url, data=row['json'])
#         print(row['json'])
#         print(x.text)
        time.sleep(delay)
end = time.time()

In [56]:
print "Send %s gps in %s secs. Avg = %.2f ms" % (total_gps, end - start, (end - start)*1000/total_gps)

Send 1222 gps in 12.0523440838 secs. Avg = 9.86 ms
